In [0]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np
#pandas- librărie pentru lucrul cu fișierele
import pandas as pd
import torch
import torch.nn as nn

In [0]:
df=pd.read_csv("/content/wine2.csv",header=None)

In [28]:
df.values

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 3.920e+00, 1.065e+03,
        0.000e+00],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 3.400e+00, 1.050e+03,
        0.000e+00],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 3.170e+00, 1.185e+03,
        0.000e+00],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 1.560e+00, 8.350e+02,
        1.000e+00],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 1.620e+00, 8.400e+02,
        1.000e+00],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 1.600e+00, 5.600e+02,
        1.000e+00]])

In [0]:
#Dataset - o clasă din PyTorch foarte utilă gestionării seturilor de date
class WineDataset(Dataset):
    """ Wine dataset."""
    # Initialize your data, download, etc.
    def __init__(self):
        #Citim setul de date
        df=pd.read_csv("/content/wine2.csv",header=None, dtype=np.float32)
        xy = torch.from_numpy(df.values)
        self.len = xy.shape[0]
        #Vom folosi ca input toate valorile mai puțin ultima coloană
        self.x_data = xy[:, 0:-1]
        #Vom folosi ca output ultima coloană
        self.y_data = xy[:, [-1]]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [0]:
dataset = WineDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=1)

In [10]:
dataset[0]

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]), tensor([0.]))

In [0]:
class Model1(nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model1, self).__init__()
        self.l1 = nn.Linear(13, 6)
        self.l2 = nn.Linear(6, 4)
        self.l3 = nn.Linear(4, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

In [0]:

class Model2(nn.Module):

    def __init__(self):
        """
        Dat fiind faptul că 
        """
        super(Model2, self).__init__()

        #Practic toate functiile liniare și non-liniare realizează o secvență a flowlui de date
        self.sequential=nn.Sequential(
            nn.Linear(13,6),
            nn.ReLU(),
            nn.Linear(6,4),
            nn.ReLU(),
            nn.Linear(4,1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.sequential(x)

In [0]:
model=Model2()

In [0]:
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

In [42]:
# Training loop
for epoch in range(200):
  #Aplicăm modificarea de learning rate per epocă, deci va trebui să avem un istoric al loss-ului
  losses=[]
  for i, data in enumerate(train_loader, 0):
      # get the inputs
      inputs, labels = data

      # Forward pass: Compute predicted y by passing x to the model
      y_pred = model(inputs)

      # Compute and print loss
      loss = criterion(y_pred, labels)
      print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')
      losses.append(loss.item())
      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  mean_loss=sum(losses)/len(losses)
  scheduler.step(mean_loss)

Epoch 1 | Batch: 1 | Loss: 65.2756
Epoch 1 | Batch: 2 | Loss: -800.0000
Epoch 1 | Batch: 3 | Loss: -1100.0000
Epoch 1 | Batch: 4 | Loss: 500.0000
Epoch 1 | Batch: 5 | Loss: 200.0000
Epoch 1 | Batch: 6 | Loss: 100.0000
Epoch 2 | Batch: 1 | Loss: -400.0000
Epoch 2 | Batch: 2 | Loss: -300.0000
Epoch 2 | Batch: 3 | Loss: 800.0000
Epoch 2 | Batch: 4 | Loss: -800.0000
Epoch 2 | Batch: 5 | Loss: -100.0000
Epoch 2 | Batch: 6 | Loss: -400.0000
Epoch 3 | Batch: 1 | Loss: -700.0000
Epoch 3 | Batch: 2 | Loss: 300.0000
Epoch 3 | Batch: 3 | Loss: 0.0000
Epoch 3 | Batch: 4 | Loss: -400.0000
Epoch 3 | Batch: 5 | Loss: -300.0000
Epoch 3 | Batch: 6 | Loss: -100.0000
Epoch 4 | Batch: 1 | Loss: -1000.0000
Epoch 4 | Batch: 2 | Loss: 100.0000
Epoch 4 | Batch: 3 | Loss: -500.0000
Epoch 4 | Batch: 4 | Loss: 500.0000
Epoch 4 | Batch: 5 | Loss: 500.0000
Epoch 4 | Batch: 6 | Loss: -800.0000
Epoch 5 | Batch: 1 | Loss: -1000.0000
Epoch 5 | Batch: 2 | Loss: 1000.0000
Epoch 5 | Batch: 3 | Loss: -400.0000
Epoch 5 | B